<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline

import numpy
import matplotlib.pyplot as plt
import matplotlib.animation

from IPython.display import HTML
from clawpack import pyclaw
from clawpack import riemann

# Finite Volume Methods for Nonlinear Systems

## Godunov's Method

Recall that our goal is to evolve the cell-averages
$$
    Q^n_i \approx \frac{1}{\Delta x} \int^{x_{i+1/2}}_{x_{i-1/2}} q(x, t_n) dx
$$
using a piecewise reconstruction $\widetilde{q}^n(x, t_n)$ using these cell-averages and evolving these functions using the conservation law.

Solving (evolving) over time $\Delta t$ with this data gives us the function $\widetilde{q}^n(x, t_{n+1})$ leading to
$$
    Q^{n+1}_i = \frac{1}{\Delta x} \int^{x_{i+1/2}}_{x_{i-1/2}} \widetilde{q}^n(x, t_{n+1}) dx
$$.

The final component of Godunov's method suggests that we do not need the entire Riemann solution but only need the solution along the cell-interface $x_{i-1/2}$ such that
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} (F^n_{i+1/2} - F^n_{i-1/2})
$$
where
$$
    F^n_{i-1/2} = \mathcal{F}(Q^n_{i-1}, Q^n_i) = f(\widehat{q}(Q^n_{i-1}, Q^n_i)
$$
where $\widehat{q}(Q^n_{i-1}, Q^n_i)$ is the Riemann solution evaluated along $x/t = 0$.

Godunov's method can also be implemented in wave-propagation form
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} (\mathcal{A}^+ \Delta Q_{i-1/2} + \mathcal{A}^- \Delta Q_{i+1/2}),
$$
which takes the fluxes to be
$$\begin{aligned}
    \mathcal{A}^- \Delta Q_{i-1/2} &= f(\widehat{Q}_{i-1/2}) - f(Q_{i-1}) \\
    \mathcal{A}^+ \Delta Q_{i-1/2} &= f(Q_{i}) - f(\widehat{Q}_{i-1/2}).
\end{aligned}$$

The primary note of importance now is that all we need for Godunov's method is what the solution along the grid cell edge is rather than the full Riemann solution we have been working with.  This strongly suggests that there may be ways to use approximate Riemann solvers that give us only what we need and are less expensive.

## Convergence of Godunov's Method

It also useful at this point to review what we know about the convergence of Godunov's method that we showed before.

1. The Lax-Wendroff theorem implies that for nonlinear systems of conservation laws, if we have a sequence of numerical approximations representing grid refinement that this sequence will converge in the appropriate sense to a function $q(x,t)$ and that this functions is a weak solution of the conservation law.  This is unfortunately not true in general for the nonconservative form of the equations.

2. Entropy conditions will allow us to pick out the correct weak solution and if we employ Riemann solvers that obey the appropriate Entropy conditions that the overall method will also pick out the entropy satisfying solution.

3. The Lax-Wendroff theorem unfortunately does not guarantee convergence, rather it only says *if* a sequence converges that it converges to a weak solution of the conservation law.  Showing convergence requires a form of stability for which we used TV-stability before.  Unfortunately TV-stability cannot be extended as is to the system case.

## Approximate Riemann Solvers

We now will start to discuss the idea that perhaps we only need a small part of the full Riemann solution if we are interested in using Godunov's methods.  In particular, if $\widehat{q}(q_\ell, q_r)$ is the general, full solution to a Riemann problem that we only need to know the state along $x/t = 0$.  This usually implies that we need to compute one of the middle states $q_m$ of the Riemann solution although this is highly dependent on wave speeds and criticality conditions.

Define a function
$$
    \widehat{Q}_{i-1/2}(x/t)
$$
that approximates the true similarity solution of the Riemann problem with input data $Q_{i-1}$ and $Q_i$.  This approximation will generally depend on some set of jumps in $Q$ where
$$
    Q_i - Q_{i-1} = \sum^{M_w}_{p=1} \mathcal{W}^p_{i-1/2}
$$
where now we are allowed to pick out how many waves $M_w$ represent the approximation.

Generalizing Godunov's method to systems then we could take two different approaches to defining the fluctuations:
1. Define the numerical flux by
$$
    F_{i-1/2} = f(\widehat{Q}_{i-1/2})
$$
where
$$
    \widehat{Q}_{i-1/2} = Q_{i-1} + \sum_{p:s^p_{i-1/2} < 0} \mathcal{W}^p_{i-1/2}.
$$
In other words the state that lies along $x/t = 0$.  We can also go the other direction so that
$$
    \widehat{Q}_{i-1/2} = Q_{i} - \sum_{p:s^p_{i-1/2} > 0} \mathcal{W}^p_{i-1/2}.
$$
Therefore the fluctuations are 
$$\begin{aligned}
    \mathcal{A}^- \Delta Q_{i-1/2} &= f(\widehat{Q}_{i-1/2}) - f(Q_{i-1}) \\
    \mathcal{A}^+ \Delta Q_{i-1/2} &= f(Q_{i}) - f(\widehat{Q}_{i-1/2}) \\
\end{aligned}$$
1. Use the waves and speeds to define
$$\begin{aligned}
    \mathcal{A}^- \Delta Q_{i-1/2} &= \sum^{M_w}_{p=1} (s^p_{i-1/2})^- \mathcal{W}^p_{i-1/2} \\
    \mathcal{A}^+ \Delta Q_{i-1/2} &= \sum^{M_w}_{p=1} (s^p_{i-1/2})^+ \mathcal{W}^p_{i-1/2} \\
\end{aligned}$$

The important realization here is that both of these approaches are the same for the all-shock Riemann solution.  This also implies that unless we have a transonic rarefaction, i.e. a rarefaction fan contains $x/t = 0$ we do not need to worry about what the exact type of wave is but rather what middle state contains $x/t = 0$.

### Linearized Riemann Solvers

Probably the most natural way to find an approximate Riemann solver is to find some linearization of the problem that is appropriate for the conservation law such that for $q_t + f(q)_x = 0$ we can instead locally solve
$$
    \widehat{q}_t + \widehat{A}_{i-1/2} \widehat{q}_x = 0
$$
such that the matrix $\widehat{A}_{i-1/2}$ as an appropriate approximation to $f'(q)$ valid in the neighborhood of $x_{i-1/2}$.  We also need to require that $\widehat{A}_{i-1/2}$ is diagonalizable with real eigenvalues so that we can have some sense that
$$
    \widehat{A}_{i-1/2} \rightarrow f'(\overline{q}) \quad \text{as } Q_{i-1}, Q_i \rightarrow \overline{q}
$$
for consistency.

One of the reasons that we expect this to work is that for most Riemann solutions the shocks are well isolated.  This implies that the difference from one cell to another
$$
    ||Q_i - Q_{i-1}|| = \mathcal{O}(\Delta x)
$$
as long as the jump is not large and therefore
$$
    f'(Q_{i-1}) \approx f'(Q_i).
$$

We also know that if $||Q_i - Q_{i-1}|| = \mathcal{O}(\Delta x)$ that we expect that the Hugoniot loci and integral curves are similar to the eigenvectors of the system.

The solution would then be similar to the linear hyperbolic systems we have studied before with the waves determined by the eigenvectors $\widehat{r}^p_{i-1/2}$ and speeds $s^p_{i-1/2} = \widehat{\lambda}^p_{i-1/2}$.  This also allows us to easily identify the waves as
$$
    Q_i - Q_{i-1} = \sum^m_{p=1} \alpha^p_{i-1/2} \widehat{r}^p_{i-1/2}
$$
and therefore
$$
    \mathcal{W}^p_{i-1/2} = \alpha^p_{i-1/2} \widehat{r}^p_{i-1/2}.
$$

There are of course multiple ways to form this linearized approximation.  In general we could use
$$
    \widehat{A}_{i-1/2} = f'(\overline{Q}_{i-1/2})
$$
where $\overline{Q}_{i-1/2}$ is some appropriate "average state" dependent on $Q_i$ and $Q_{i-1}$.

 - What "average state" would you propose?
 - What properties of the solution might not work in general?

The most obvious average is the true average of the values
$$
    \overline{Q}_{i-1/2} = \frac{1}{2} (Q_i + Q_{i-1}).
$$
Although this is consistent this does not imply that the method is consistent unless some form of 
$$
    \widehat{Q}_{i-1/2} = Q_{i-1} + \sum_{p:s^p_{i-1/2} < 0} \mathcal{W}^p_{i-1/2}
$$
and 
$$\begin{aligned}
    \mathcal{A}^- \Delta Q_{i-1/2} &= f(\widehat{Q}_{i-1/2}) - f(Q_{i-1}) \\
    \mathcal{A}^+ \Delta Q_{i-1/2} &= f(Q_{i}) - f(\widehat{Q}_{i-1/2}) \\
\end{aligned}$$
are satisfied.  Unfortunately
$$\begin{aligned}
    \mathcal{A}^- \Delta Q_{i-1/2} &= \sum^{M_w}_{p=1} (s^p_{i-1/2})^- \mathcal{W}^p_{i-1/2} \\
    \mathcal{A}^+ \Delta Q_{i-1/2} &= \sum^{M_w}_{p=1} (s^p_{i-1/2})^+ \mathcal{W}^p_{i-1/2} \\
\end{aligned}$$
does not guarantee conservation unless we add an additional condition.

The primary condition we need to ensure for conservation is actually
$$
    f(Q_i) - f(Q_{i-1}) = \sum^{M_w}_{p=1} s^p_{i-1/2} \mathcal{W}^p_{i-1/2},
$$
which in general is not satisfied for many different forms of $\overline{Q}_{i-1/2}$.

Averaging $Q$ values is not the only approach, why not average the flux values with
$$
    \widehat{A}_{i-1/2} = \frac{1}{2}[f'(Q_{i-1}) + f'(Q_i)]
$$
or perhaps some other average of Jacobian evaluations.  Unfortunately this also does not satisfy the jump in fluxes previously mentioned unless care is taken.

### Roe Linearization

One of the keys to providing a robust linearization is to put some conditions on the linearization and its eigenspace.  The first of these is:

> If $Q_{i-1}$ and $Q_i$ are connected by a single wave $\mathcal{W}^p = Q_i - Q_{i-1}$ in the true Riemann solution, then $\mathcal{W}^p$ should also be an eigenvector of $\widehat{A}_{i-1/2}$.

If this is true then the approximation will consist of a single wave that agrees with the exact Riemann solution with the strongest solution.

Another way to say this is that if $Q_i$ and $Q_{i-1}$ are connected by a single wave, then
$$
    f(Q_i) - f(Q_{i-1}) = s (Q_i - Q_{i-1}).
$$
If the linearized problem also has this form then
$$
    \widehat{A}_{i-1/2} (Q_i - Q_{i-1}) = s (Q_i - Q_{i-1})
$$
implying
$$
    \widehat{A}_{i-1/2} (Q_i - Q_{i-1}) = f(Q_i) - f(Q_{i-1}).
$$
If this last expression is true then an approximate solver of this form is in fact conservative.

This can also be shown via
$$
    \mathcal{A}^- \Delta Q_{i-1/2} + \mathcal{A}^+ \Delta Q_{i-1/2} = f(Q_i) - f(Q_{i-1}),
$$
which is implied by the above condition.  Consequently the condition
$$
    \widehat{A}_{i-1/2} (Q_i - Q_{i-1}) = f(Q_i) - f(Q_{i-1})
$$
is often called **Roe's Condition**.

The practical side of this is that we need to find an average that satisfies this condition.  One way to do this is to think of the problem as finding a path through state space connecting $Q_i$ and $Q_{i-1}$ parameterized by
$$
    q(\xi) = Q_{i-1} + (Q_i - Q_{i-1}) \xi
$$
for $\xi \in [0, 1]$ and require it satisfy Roe's condition.

Writing this out we then have
$$\begin{aligned}
    f(Q_i) - f(Q_{i-1}) &= \int^1_0 \frac{\text{d}}{\text{d} \xi} f(q(\xi)) d\xi \\
    &= \int^1_0 f'(q(\xi)) q'(\xi) d\xi \\
    &= \left[ \int^1_0 f'(q(\xi)) d\xi \right ] (Q_i - Q_{i-1}).
\end{aligned}$$

Recalling that we need 
$$
    \widehat{A}_{i-1/2} (Q_i - Q_{i-1}) = f(Q_i) - f(Q_{i-1})
$$
this implies that 
$$
    f(Q_i) - f(Q_{i-1}) = \left[ \int^1_0 f'(q(\xi)) d\xi \right ] (Q_i - Q_{i-1})
$$
gives us
$$
    \widehat{A}_{i-1/2} = \int^1_0 f'(q(\xi)) d\xi.
$$

This unfortunately does not guarantee that the resulting matrix $\widehat{A}_{i-1/2}$ is diagonalizable with real eigenvalues.  The integral itself can also be difficult to evaluate leaving us wanting a better approach.

Instead Roe proposed a **parameter vector** $z(q)$, effectively a change of variables, that leads not only to easier evaluation of the integrals but also to evaluations that satisfies properties that we want.

Here we now will integrate along the path
$$
    z(\xi) = Z_{i-1} + (Z_i - Z_{i-1}) \xi
$$
where $Z_j = z(Q_j)$ for $j=i-1, i$ and therefore $z'(\xi) = Z_i - Z_{i-1}$ that is independent of $\xi$.  This implies
$$\begin{aligned}
    f(Q_i) - f(Q_{i-1}) &= \int^1_0 \frac{\text{d}}{\text{d} \xi} f(z(\xi)) d\xi \\
    &= \int^1_0 f'(z(\xi)) z'(\xi) d\xi \\
    &= \left[ \int^1_0 f'(z(\xi)) d\xi \right ] (Z_i - Z_{i-1}).
\end{aligned}$$

This expression we hope is easier to evaluate but we have no idea what this expression $z(\xi)$ really is yet.  We can find this by rewriting $z(q)$ as
$$\begin{aligned}
    f(Q_i) - f(Q_{i-1}) &= \widehat{C}_{i-1/2} (Z_i - Z_{i-1}) \\
    Q_i - Q_{i-1} &= \widehat{B}_{i-1/2} (Z_i - Z_{i-1})
\end{aligned}$$
and therefore observing
$$
    \widehat{A}_{i-1/2} = \widehat{C}_{i-1/2} \widehat{B}^{-1}_{i-1/2}.
$$

Harten and Lax showed that this approach will always be able to produce $\widehat{A}_{i-1/2}$ if the system has a convex entropy.  One can actually also then choose $z(q) = \eta'(q)$.

This being true we still want to ensure that the integrals of interest are easily evaluated, which is best shown by an example. 

#### Example:  Roe Solver for Shallow Water

$$
    q = \begin{bmatrix} h \\ hu \end{bmatrix} = \begin{bmatrix} q^1 \\ q^2 \end{bmatrix} \quad f(q) = \begin{bmatrix} hu \\ hu^2 + \frac{1}{2} gh^2 \end{bmatrix} = \begin{bmatrix} q^2 \\ \frac{q^2}{(q^1)^2} + \frac{1}{2} g (q^1)^2 \end{bmatrix}
$$
and
$$
    f'(q) = \begin{bmatrix}
        0 & 1 \\
        -\left(\frac{q^2}{q^1} \right)^2 + g q^1 & 2 \frac{q^2}{q^1}
    \end{bmatrix} = \begin{bmatrix}
        0 & 1 \\
        -u^2 + g h & 2 u
    \end{bmatrix}
$$

Choose the parameterization as
$$
    z = h^{-1/2} q \quad \Rightarrow \quad \begin{bmatrix} z^1 \\ z^2 \end{bmatrix} = \begin{bmatrix} \sqrt{h} \\ u \sqrt{h} \end{bmatrix}
$$

See if you can carry this parameterization though and find $\widehat{A}_{i-1/2}$.

Taking
$$
q(z) = \begin{bmatrix}
    (z^1)^2 \\ z^1 z^2
    \end{bmatrix} \quad \Rightarrow \quad 
    \frac{\partial q}{\partial z} = \begin{bmatrix}
        2z^1 & 0 \\
        z^2 & z^1
    \end{bmatrix}
$$
and therefore
$$
    f(z) = \begin{bmatrix}
        z^1 z^2 \\
        (x^2)^2 + \frac{1}{2} g (z^1)^4
    \end{bmatrix} \quad \Rightarrow \quad \frac{\partial}{\partial z} f(z) = \begin{bmatrix}
        z^2 & z^1 \\
        2 g (z^1)^3 & 2 z^2
    \end{bmatrix}
$$

We now need to integrate from $\xi = 0 \ldots 1$ where
$$
    z^p = Z^p_{i-1} + (Z^p_i - Z^p_{i-1}) \xi.
$$
At this point all of our traverses through state space are linear except for one but we are still considering polynomials.

Integrating the linear terms in our integrals leads us to
$$
    \int^1_0 z^p(\xi) d\xi = \frac{1}{2} (Z^p_{i-1} + Z^p_{i}) \equiv \overline{Z}^p,
$$
clearly just the average of the states of the transformed quantities $z(q)$.

Integrating the higher order terms we have
$$\begin{aligned}
    \int^1_0 (z^1(\xi))^3 d\xi &= \frac{1}{4} \left( \frac{(Z^1_i)^4 - (Z^1_{i-1})^4}{Z^1_i - Z^1_{i-1}} \right) \\
    &= \frac{1}{2}(Z^1_{i-1} + Z^1_i) \cdot \frac{1}{2} \left [ (Z^1_{i-1})^2 + (Z^1_i)^2 \right ] \\
    &= \overline{Z}^1 \overline{h},
\end{aligned}$$
where
$$
    \overline{h} = \frac{1}{2} (h_{i-1} + h_i).
$$

From this we obtain
$$
    \widehat{B}_{i-1/2} = \begin{bmatrix}
        2 \overline{Z}^1 & 0 \\
        \overline{Z}^2 & \overline{Z}^1
    \end{bmatrix}
$$
and
$$
    \widehat{C}_{i-1/2} = \begin{bmatrix}
        \overline{Z}^2 & \overline{Z}^1 \\
        2 g \overline{Z}^1 \overline{h} & 2 \overline{Z}^2
    \end{bmatrix}.
$$

$$
    \widehat{B}_{i-1/2} = \begin{bmatrix}
        2 \overline{Z}^1 & 0 \\
        \overline{Z}^2 & \overline{Z}^1
    \end{bmatrix} \quad \quad
    \widehat{C}_{i-1/2} = \begin{bmatrix}
        \overline{Z}^2 & \overline{Z}^1 \\
        2 g \overline{Z}^1 \overline{h} & 2 \overline{Z}^2
    \end{bmatrix}
$$

Therefore
$$
    \widehat{A}_{i-1/2} = \widehat{C}_{i-1/2} \widehat{C}^{-1}_{i-1/2} = \begin{bmatrix}
        0 & 1 \\
        -\left(\frac{\overline{Z}^2}{\overline{Z}^1} \right)^2 + g \overline{h} & 2 \frac{\overline{Z}^2}{\overline{Z}^1}
    \end{bmatrix} = \begin{bmatrix}
        0 & 1\\
        -\widehat{u}^2 + g \overline{h} & 2 \widehat{u}
    \end{bmatrix}
$$
where
$$
    \overline{h} = \frac{1}{2} (h_{i-1} + h_i)
$$
and
$$
    \widehat{u} = \frac{\overline{Z}^2}{\overline{Z}^1} = \frac{u_{i-1} \sqrt{h_{i-1}} + u_i \sqrt{h_i}}{\sqrt{h_{i-1}} + \sqrt{h_i}}
$$

### Sonic Entropy Fixes

One of the biggest drawbacks to a Roe linearized Riemann solver is that the solution formally only consists of shocks.

Even in the scalar case the Roe condition can be satisfied by
$$
    \widehat{\mathcal{A}}_{i-1/2} = \frac{f(Q_i) - f(Q_{i-1})}{Q_i - Q_{i-1}}
$$
where here $\widehat{\mathcal{A}}_{i-1/2}$ is a scalar.  This is of course the shock speed.

As mentioned before numerically this is only a problem for transonic rarefactions where
$$
    f'(q_\ell) < 0 < f'(q_r)
$$
for the scalar case (these are of course the edges of the rarefaction wave).
The same holds true for systems of equations when a particular wave is a transonic rarefaction.

For the shallow water equations we can easily check if one of the two waves is a transonic rarefaction with the following computation:
$$\begin{aligned}
    \lambda^1_{i-1} = u_{i-1} - \sqrt{g h_{i-1}} & & \lambda^1_m = u_m - \sqrt{g h_m} \\
    \lambda^2_{m} = u_{m} - \sqrt{g h_{m}} & & \lambda^2_i = u_i - \sqrt{g h_i}.
\end{aligned}$$
Similar to the previous condition if any of these values in a row are separated by zero then we know we have a transonic rarefaction.

The biggest impediment to using these conditions is that we need to know $q_m$.

For simple systems this may not be too hard a burden as we know that if there is a transonic rarefaction there can be only one.  Assuming there is one we can use the simplification that we need to know $\xi = x/t = 0$.  For instance in the 1-rarefaction case we know
$$\begin{aligned}
    \widehat{h}_{i-1/2} &= \frac{\left(u_{i-1} + 2 \sqrt{g h_{i-1}} \right)^2}{9g} \\
    \widehat{u}_{i-1/2} &= u_{i-1} - 2 \left( \sqrt{g h_{i-1}} - \sqrt{g \widehat{h}_{i-1/2}} \right)
\end{aligned}$$

#### Harten-Hyman Entropy Fix

A easier and more general approach to entropy fixes is due to Harten and Hyman and is generally the approach used in many Clawpack solvers.

The principle approach is this, suppose that a transonic rarefaction exists in the $k$-family and therefore 
$$
    \lambda^k_\ell < 0 < \lambda^k_r
$$ 
and with
$$\begin{aligned}
    q^k_\ell &= Q_{i-1} + \sum^{k-1}_{p=1} \mathcal{W}^p \\
    q^k_r &= q_\ell^k + \mathcal{W}^k,
\end{aligned}$$
in other words the state to the left and right of the rarefaction.

Now replace the single wave $\mathcal{W}^k$ propagating with speed $\widehat{\lambda}^l$ by two waves
$$
    \mathcal{W}^k_\ell = \beta \mathcal{W}^k \quad \mathcal{W}^k_r = (1 - \beta) \mathcal{W}^k
$$
propagating at speeds $\lambda^k_\ell$ and $\lambda^k_r$ respectively.  Maintaining conservation requires
$$
    \lambda^l_\ell \mathcal{W}^k_\ell + \lambda^k_r \mathcal{W}^k_r = \widehat{\lambda}^k \mathcal{W}^k
$$
and therefore
$$
    \beta = \frac{\lambda^k_r - \widehat{\lambda}^k}{\lambda^k_r - \lambda^k_\ell}.
$$
This amounts to splitting the wave into two pieces traveling to the left and right and therefore modifying the fluctuations $\mathcal{A}^\pm \Delta Q$.

#### Numerical Viscosity

One way to view the entropy problem as mentioned before is that not enough viscosity is being introduced into the solution.  Numerical viscosity can solve this for us and we can modify Roe's linearization to account for this.

The numerical flux for Roe's method is
$$\begin{aligned}
    F_{i-1/2} &= \frac{1}{2} [f(Q_{i-1}) + f(Q_i) ] - \frac{1}{2} \left | \widehat{A}_{i-1/2} \right | (Q_i - Q_{i-1}) \\
    &=\frac{1}{2} [f(Q_{i-1}) + f(Q_i) ] - \frac{1}{2} \sum_p \left | \widehat{\lambda}^p_{i-1/2} \right | \mathcal{W}^p_{i-1/2}.
\end{aligned}$$
The sum in the last expression can be looked upon as being a form of viscosity.

If a transonic rarefaction is present then we expect that one of the eigenvalues $\widehat{\lambda}^p_{i-1/2}$ is very close to zero and the corresponding term in the last sum will see very little viscosity.  This is in fact often what we observe, a stationary shock where there should be none since the corresponding speed is identically zero.

In [ ]:
def true_solution(x, t):
    if t > 0:
        t_vec = t * numpy.ones(x.shape)
        return (x < 0) * -numpy.ones(x.shape) + \
           (-t_vec < x) * (x <= 0) * (x / t_vec + 1) + \
           (0 <= x) * (x <= 2*t_vec) * x / t_vec + \
           (2 * t_vec <= x) * 2.0 * numpy.ones(x.shape)
    else:
        return (x < 0) * -numpy.ones(x.shape) + \
           (0.0 <= x) * 2.0 * numpy.ones(x.shape)

def burgers_animation(order=2, efix=True):
   
    solver = pyclaw.ClawSolver1D(riemann.burgers_1D_py.burgers_1D)
    solver.kernel_language = "Python"
    
    solver.limiters = pyclaw.limiters.tvd.MC
    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    solver.order = order

    x = pyclaw.Dimension(-3.0, 3.0, 50, name='x')
    domain = pyclaw.Domain(x)
    num_eqn = 1
    state = pyclaw.State(domain, num_eqn)
    xc = domain.grid.x.centers
    state.q[0,:] = (xc < 0) * -numpy.ones(xc.shape) + 2.0 * (xc >= 0) * numpy.ones(xc.shape)
    state.problem_data['efix'] = efix

    claw = pyclaw.Controller()
    claw.tfinal = 1.0
    claw.num_output_times   = 10
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver

    claw.keep_copy = True
    claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    
    fig = plt.figure()
    axes = plt.subplot(1, 1, 1)
    axes.set_xlim((x[0], x[-1]))
    axes.set_ylim((-1.5, 2.5))
    axes.set_title("Burgers Equation")
    
    def init():
        axes.set_xlim((x[0], x[-1]))
        axes.set_ylim((-1.5, 2.5))
        computed_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'bo-')
        true_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'k-')
        return (computed_line, true_line)
    
    computed_line, true_line = init()
    
    def fplot(n):
        computed_line.set_data([x,], [claw.frames[n].q[0, :]])
        true_line.set_data([x,], [true_solution(x, claw.frames[n].t)])
        return (computed_line, true_line)

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=init, repeat=False)

In [ ]:
HTML(burgers_animation(order=1, efix=False).to_jshtml())

If we implement the above idea instead using the wave-propagation formulation and $\mathcal{A}^\pm \Delta Q$ we get an additional detail with the numerical flux written as
$$
    F_{i-1/2} = \frac{1}{2} [f(Q_{i-1}) + f(Q_i) ] - \frac{1}{2} \sum_p \left [ (\widehat{\lambda}^p_{i-1/2})^+ - (\widehat{\lambda}^p_{i-1/2})^- \right ] \mathcal{W}^p_{i-1/2}
$$
that allows us to apply the Harten-Hyman entropy fix.

#### Harten's Entropy Fix

Another entropy fix proposed by Harten is based on increasing the viscosity only by modifying the field that contains the eigenvalue that may be too close to zero.  This follows that we replace $|\widehat{\lambda}^p_{i-1/2})|$ by a limited value
$$
    \phi_\delta(\widehat{\lambda}^p_{i-1/2}))
$$
where
$$
    \phi_\delta(\lambda) = \left \{ \begin{aligned}
    &|\lambda| & & \text{if } \lambda \geq \delta \\
    &\frac{\lambda^2 + \delta^2}{2 \delta} & & \text{if } \lambda < \delta
    \end{aligned} \right .
$$
which effectively changes the absolute value in the original Roe flux to be perturbed from zero.  Unfortunately this approach requires tuning the parameter $\delta$ for each problem.

### Failure of Linearized Solvers

### HLL and HLLE Solvers

## High-Resolution Methods

## Alternative Wave-Propagation Implementations for Approximate Riemann Solvers

## Second-Order Accuracy

## Total Variation for Systems

### Total Variation Estimates for Linear Systems

### Wave-Strength Estimates for Nonlinear Systems

### Glimm's Random-Choice Method

### Oscillation in Shock Computations

#### Start-Up Errors

#### Slow-Moving Shocks